In [3]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:

from datasets import Dataset, concatenate_datasets  
import sys
sys.path.insert(0, "/home/sdl/SDL-LNP/model")
import random 
from unimol.lmdb_dataset import collate_fn, load_dataset, compute_hash2smi
from unimol.hf_unimol import UniMol, UniMolConfig, init_unimol_backbone


dict_path = "/home/sdl/SDL-LNP/model/evaluation/notebooks/dict.txt"
weight_path = "/home/sdl/3d_molecule_save/baseline/sdl-proj-head-ckpt-2/checkpoint_last.pt"

In [5]:
model_backbone, dictionary = init_unimol_backbone(weight_path, dict_path=dict_path)

topk_conformer = 11
lmdb_dir = "/home/sdl/vector_cellxgene_data/1920-lib"
train_data = load_dataset(
    dictionary,
    str(lmdb_dir + "/" + "train.lmdb"),
    "train",
    topk_conformer=topk_conformer,
)
valid_data = load_dataset(
    dictionary,
    str(lmdb_dir + "/" + "valid.lmdb"),
    "valid",
    topk_conformer=topk_conformer,
)
test_data = load_dataset(
    dictionary,
    str(lmdb_dir + "/" + "test.lmdb"),
    "test",
    topk_conformer=topk_conformer,
)


hf_train_data = Dataset.from_generator(
    lambda: train_data,
)

hf_valid_data = Dataset.from_generator(
    lambda: valid_data,
)

hf_test_data = Dataset.from_generator(
    lambda: test_data,
)
combined_data = concatenate_datasets([hf_train_data, hf_valid_data, hf_test_data])

Unimol backbone got unkonwn args:  ['--f=/home/sdl/.local/share/jupyter/runtime/kernel-v2-267723905K3JzIFiGAT.json']
Model loaded
Loading dataset, selecting top 11 conformers.
Loading dataset, selecting top 11 conformers.
Loading dataset, selecting top 11 conformers.


In [6]:
len(combined_data)

21120

In [13]:
hf_test_data[1]['src_tokens']

[1,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 6,
 6,
 4,
 4,
 4,
 4,
 4,
 5,
 4,
 4,
 6,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 4,
 4,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 2]

In [72]:
from core.data_manager import DataManager

# total 1920 data
all_indices = list(range(0, 1920))

# randomly select 1728 data for training
random.shuffle(all_indices)
train_indices = all_indices[:1728]
test_indices = all_indices[1728:]


data_manager = DataManager(
    combined_data,
    train_indices = train_indices,
    test_indices = test_indices,
    # label_attr = "target",
    labelled_indices=[]
)

In [73]:
data_manager.get_unlabelled_set()

Dataset({
    features: ['src_tokens', 'src_coord', 'src_distance', 'src_edge_type', 'target', 'smi_name', 'smi_string'],
    num_rows: 19008
})

In [74]:
print(data_manager)

DataManager
Training set size: 1728 molecules (19008 conformers).
Labelled: 0, Unlabelled: 1728
Percentage Labelled: 0.000


In [75]:
from core.oracle import BenchmarkOracle

benchmark_oracle = BenchmarkOracle()


In [77]:
benchmark_oracle.query_target_value(data_manager, 1)

1.038636849